## Import Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
% pip install wandb

In [2]:
# Import dependencies

from os import listdir, remove, makedirs
from os.path import join, isdir, isfile
import numpy as np
from skimage import io
from skimage.transform import resize
import random
import tensorflow as tf
from tensorflow.python.ops import variable_scope as vs
import tensorflow.keras.layers as layers

In [3]:
# Import python files

from utils import HGN_organize_data, HGN_divide_train_test
from code_HGN_senEM_train import train
from preprocess4HGN import preprocess_speech2pickup

## Load Preprocessed Data

In [4]:
relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/data_v1.2_single_channel/preprocessed4HGN_speech2pickup.npz'
    
data = np.load(relative_data_directory_path)

img_idx = data['img_idx']
sen_len = data['seq_len']
speech_inputs = data['inputs']
pos_outputs = data['outputs']
sentence = data['sentence']

print('End loading MetaData')

End loading MetaData


## Load data and divide training and test set

In [8]:
num_data = img_idx.shape[0]

img_path = '/content/drive/MyDrive/Speech2Pickup/train_img'
script_path = '/content/drive/MyDrive/Speech2Pickup/train_heatmap'

total_images, total_heatmaps = HGN_organize_data(img_path, script_path, model_type='speech2pickup');

args = dict()
args['img_idx'] = img_idx
args['speech_inputs'] = speech_inputs
args['pos_outputs'] = pos_outputs
args['num_data'] = num_data

idx_train, idx_test, num_train, num_test, \
train_img_idx, train_speech_inputs, train_pos_outputs, \
test_img_idx, test_speech_inputs, test_pos_outputs = \
HGN_divide_train_test(args, model_type='speech2pickup')

KeyboardInterrupt: ignored

## Start Training

In [7]:
batch_size = 8
max_epoch = 300
n_mels = 40
time_steps = 303
num_hg_Depth = 4
dim_hg_feat = 256
learning_rate = 0.00001
dropout_rate = 0.0
train_model_number = 1
input_channel_type = 'single'
run_date = '2020-12-07'
encoder_args = {'num_stacks': 3, 'num_channels':[n_mels for i in range(6)], 'kernel_size':3,
                'dropout_rate': dropout_rate, 'activation': 'leaky-relu', 'return_type': 'end'}
input_shapes = (None, n_mels, time_steps)
seed = 1
training_state = True

wandb_config = {'batch_size': batch_size,
                'max_epoch': max_epoch,
                'n_mels': n_mels,
                'num_hg_Depth': num_hg_Depth,
                'dim_hg_feat': dim_hg_feat,
                'learning_rate': learning_rate,
                'dropout_rate': dropout_rate,
                'train_model_number': train_model_number,
                'input_channel_type': input_channel_type,
                'seed': seed,
                'training_state': training_state,
                'num_train': num_train,
                'num_test': num_test,
                'date': run_date}
# wandb_run = wandb.init(project='Speech2Pickup', name='sentenceEM', config=wandb_config)
from code_HGN_senEM_train import train
train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      num_hg_Depth = num_hg_Depth, # good be fixed
      dim_hg_feat = dim_hg_feat, # good be fixed
      n_mels = n_mels,
      time_steps = time_steps,
      encoder_model_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/model.ckpt',
      encoder_args = encoder_args,
      input_shapes = input_shapes,
      seed = seed,
      training_state = training_state,
      batch_size = batch_size, 
      max_epoch = max_epoch,
      num_train = num_train,
      save_stride = 10,
      learning_rate = learning_rate, # good be fixed
      dropout_rate = dropout_rate,
      restore_flag = 0,
      restore_path = '',
      restore_epoch = 0,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_speech_inputs = train_speech_inputs,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs)

NameError: ignored

In [ ]:
variables_names = [v for v in tf.trainable_variables() if v.name.split('/')[0] == 'temp_convnet']
tf.train.Saver(var_list=variables_names)

In [8]:
tf.reset_default_graph()